不同於Checkpoint在還原的時候還需要先建置一個完成的架構才有辦法還原相關權重參數(keras的callback function是保留完整模型資訊)，當我們要佈署正式環境的時候，當然是希望模型拿了就用，而不是還先弄一個空架構，然後還原權重參數。TensorFlow 2.0提供SavedModel的保存格式，完整的保存整個模型的信息，對佈署到Serving、Lite、js都非常實用。

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
tf.__version__

'2.0.0'

指定硬體資源，相關可[參考](https://hackmd.io/@shaoeChen/ryWIV4vkL)

In [4]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')

In [5]:
gpus 

[PhysicalDevice(name=u'/physical_device:GPU:0', device_type=u'GPU'),
 PhysicalDevice(name=u'/physical_device:GPU:1', device_type=u'GPU')]

In [6]:
tf.config.experimental.set_visible_devices(devices=gpus[1], device_type='GPU')

In [7]:
tf.config.experimental.set_memory_growth(device=gpus[1], enable=True)

取得MNIST資料集並做標準化。

In [8]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.expand_dims(x_train / 255., -1)
x_test = np.expand_dims(x_test / 255., -1)

In [9]:
x_train.shape, x_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

將資料集與標籤做為參數提供給`tf.data`

In [10]:
datasets = tf.data.Dataset.from_tensor_slices((x_train, y_train))

In [11]:
datasets

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float64, tf.uint8)>

In [12]:
datasets = datasets.shuffle(buffer_size=128, seed=10).batch(128).repeat()

利用標準的keras Sequential來建置模型

In [12]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(filters=6, kernel_size=(5, 5), padding='valid', activation='tanh'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 5), padding='valid', activation='tanh'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='tanh'),
    tf.keras.layers.Dense(84, activation='tanh'),
    tf.keras.layers.Dense(10, activation='softmax'),
])

確認模型

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 6)         156       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 16)          2416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               30840     
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

編譯模型

In [14]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy']
)

訓練模型

In [15]:
%%time
model.fit(datasets,
          epochs=5, 
          steps_per_epoch=int(len(x_train)/128))

Train for 468 steps
Epoch 1/5
468/468 [==============================] - 5s 10ms/step - loss: 1.0186 - accuracy: 0.7540
Epoch 2/5
468/468 [==============================] - 3s 6ms/step - loss: 0.3165 - accuracy: 0.9176
Epoch 3/5
468/468 [==============================] - 3s 6ms/step - loss: 0.2098 - accuracy: 0.9421
Epoch 4/5
468/468 [==============================] - 3s 6ms/step - loss: 0.1602 - accuracy: 0.9549
Epoch 5/5
468/468 [==============================] - 3s 6ms/step - loss: 0.1312 - accuracy: 0.9629
CPU times: user 18.8 s, sys: 2.06 s, total: 20.9 s
Wall time: 15.3 s


接下來就可以利用`saved_model`來保存模型，其參數為指定路徑而不是檔案名稱，要特別注意

In [17]:
tf.saved_model.save(obj=model, export_dir='model/1')

INFO:tensorflow:Assets written to: model/1/assets


執行之後在相對應的目錄上就會擁有一個`saved_model.pb`檔

參考[官方文檔](https://www.tensorflow.org/api_docs/python/tf/saved_model/load?version=stable)

這個pb檔就包含著權重與推導圖，只要載入之後就可以直接應用

In [18]:
new_model = tf.saved_model.load('model/1')

預設情況下，你的signature key為'serving_default'，但如有需求在保存的時候就需要另外指定，後續會有案例說明

In [62]:
new_model.signatures.keys()

[u'serving_default']

取得signatures function

In [65]:
infer = new_model.signatures['serving_default']
print(infer)

這邊只吃tensor而不吃numpy，而input_1這個名稱則每個人的模型可能不一樣，總之你輸入錯誤的時候訊息自然會跟你說你的input名稱了

In [88]:
print(infer(input_1=tf.constant(np.expand_dims(x_train[0].astype('float32'), 0))))

{u'dense_2': <tf.Tensor: id=11382, shape=(1, 10), dtype=float32, numpy=
array([[6.0728920e-04, 1.5178646e-04, 4.1146576e-03, 2.5311598e-01,
        3.0360779e-06, 7.3566568e-01, 1.9981069e-04, 1.0613516e-03,
        3.3403984e-03, 1.7399758e-03]], dtype=float32)>}


如果你吃numpy的話，會出現錯誤訊息，不過錯誤訊息太長，為了不影響閱讀不就測試，有興趣可以自己測試一下。

In [ ]:
# 我會報錯，was not Tensor
print(infer(input_1=np.expand_dims(x_train[0].astype('float32'), 0)))

這邊驗證與原始模型所得的output是一致的

In [91]:
model.predict(np.expand_dims(x_train[0], 0))

array([[6.0728920e-04, 1.5178646e-04, 4.1146576e-03, 2.5311598e-01,
        3.0360779e-06, 7.3566568e-01, 1.9981069e-04, 1.0613516e-03,
        3.3403984e-03, 1.7399758e-03]], dtype=float32)

如果是用`tf.keras.Model`來建置的類別模型的話，那在建構模型的時候就要在`call`的部份用`@tf.function`來裝飾

restart專案之後開始下面的測試

In [14]:
class LeNet5(tf.keras.Model):
    def __init__(self):
        # 一定要繼承父類的__init__才能使用父類相關的method與attribute
        super(LeNet5, self).__init__()
#         self.input = tf.keras.layers.InputLayer(input_shape=(28, 28, 1))
        self.conv1 = tf.keras.layers.Conv2D(filters=6, kernel_size=(5, 5), padding='valid', activation='tanh')
        self.maxpool1 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
        self.conv2 = tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 5), padding='valid', activation='tanh')
        self.maxpool2 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(120, activation='tanh')
        self.dense2 = tf.keras.layers.Dense(84, activation='tanh')
        self.pred_y = tf.keras.layers.Dense(10, activation='softmax')
    
    @tf.function
    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return self.pred_y(x)

In [15]:
model =LeNet5()

In [16]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy']
)

In [17]:
%%time
model.fit(datasets,
          epochs=5, 
          steps_per_epoch=int(len(x_train)/128))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Train for 468 steps
Epoch 1/5
468/468 [==============================] - 5s 10ms/step - loss: 1.0600 - accuracy: 0.7480
Epoch 2/5
468/468 [==============================] - 3s 6ms/step - loss: 0.3329 - accuracy: 0.9117
Epoch 3/5
468/468 [==============================] - 3s 6ms/step - loss: 0.2202 - accuracy: 0.9390
Epoch 4/5
468/468 [==============================] - 3s 6ms/step - loss: 0.1651 - accuracy: 0.9533
Epoch 5/5
468/468 [==============================] - 2s 5ms/step - loss: 0.1326 - accuracy:

In [18]:
tf.saved_model.save(obj=model, export_dir='model/2')


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model/2/assets


一樣的方法來取回模型

In [20]:
new_model = tf.saved_model.load('model/2')

要特別注意的是，不管是什麼方式，只要用saved_model的方式保存都會失去自我，不再是原始的模型類別

In [21]:
type(new_model)

tensorflow.python.saved_model.load._UserObject

In [22]:
type(model)

__main__.LeNet5

不特定設置的話，就是`serving_default`

In [23]:
new_model.signatures.keys()

[u'serving_default']

In [24]:
infer = new_model.signatures['serving_default']

In [26]:
print(infer(input_1=tf.constant(np.expand_dims(x_train[0], 0))))

{u'output_1': <tf.Tensor: id=11341, shape=(1, 10), dtype=float32, numpy=
array([[2.7317894e-04, 1.9750377e-04, 1.7206810e-03, 1.4695407e-01,
        7.9958477e-07, 8.4847361e-01, 1.3325976e-04, 9.2332449e-04,
        8.5320004e-04, 4.7043379e-04]], dtype=float32)>}


In [29]:
model.predict(np.expand_dims(x_train[0], 0))

array([[2.7317894e-04, 1.9750377e-04, 1.7206810e-03, 1.4695407e-01,
        7.9958477e-07, 8.4847361e-01, 1.3325976e-04, 9.2332449e-04,
        8.5320004e-04, 4.7043379e-04]], dtype=float32)

或者直接餵入資料即可

In [35]:
new_model((np.expand_dims(x_train[0], 0)))

<tf.Tensor: id=11452, shape=(1, 10), dtype=float32, numpy=
array([[2.7317894e-04, 1.9750377e-04, 1.7206810e-03, 1.4695407e-01,
        7.9958477e-07, 8.4847361e-01, 1.3325976e-04, 9.2332449e-04,
        8.5320004e-04, 4.7043379e-04]], dtype=float32)>

當然，原始keras的api就支援直接保存了，不過得到的是hdf5格式要注意一下就是了，而且只支援function model或sequential建立的模型，如果是`tf.keras.Model`的模型是無法使用，要特別注意

In [ ]:
# 就這麼簡單
model.save('your_file_path.hdf5')
tf.keras.models.load_model('your_file_path.hdf5')